In [1]:
import sys
import time
from wideboost.wrappers import wxgb
import numpy as np
import xgboost as xgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=False,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    
for i in ds_test.batch(60000):
    b = i

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

In [3]:
def _onehot(Y):
    b = np.zeros((Y.shape[0], Y.max().astype(int)+1))
    b[np.arange(Y.shape[0]),Y.astype(int).flatten()] = 1
    return b

In [4]:
def get_data(seed):
    np.random.seed(seed)
    train_idx = np.random.choice(range(xtrain.shape[0]), round(xtrain.shape[0] * 0.7), replace=False)
    valid_idx = np.array(list(set(range(xtrain.shape[0])) - set(train_idx)))
    train_idx = np.random.choice(train_idx, 3_000, replace=False)
    valid_idx = np.random.choice(valid_idx, 2_000, replace=False)

    test_idx = np.random.choice(range(xtest.shape[0]), 1_000, replace=False)

    features = np.random.choice(range(xtrain.shape[1]), 200, replace=False)


    y_train = _onehot(ytrain[train_idx])
    y_valid = _onehot(ytrain[valid_idx])
    y_test =  _onehot(ytest[test_idx])

    return (
        xtrain[train_idx[:,None], features],
        xtrain[valid_idx[:,None], features],
        xtest[test_idx[:,None], features], y_train, y_valid, y_test
    )

In [5]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval
import os

iters = 30
results = []
for est in range(iters):

    t0 = time.time()
    x_train, x_valid, x_test, y_train, y_valid, y_test = get_data(1000 + est)
    watchlist = [
        ((x_train, y_train),'train'),
        ((x_test, y_test), 'test'),
        ((x_valid, y_valid), 'valid')
    ]

    best_val = 1000.0
    xgb_test_val = 1000.0
    nrounds = 0
    xgb_param = {}

    def objective(param):
        global best_val
        global xgb_test_val
        global xgb_param
        global nrounds
        num_round = 500
        param['output_dim'] = 10
        param['nthread'] = 2
        print(param)

        xgb_results = {}
        wbst = wxgb.fit(
            x_train, y_train,
            param, num_round, watchlist,
            evals_result=xgb_results,
            early_stopping_rounds=20,
            verbose_eval=200
        )
        
        if (min(xgb_results['valid']['many_logloss'])) < best_val:
            print("NEW BEST VALUE!")
            best_val = min(xgb_results['valid']['many_logloss'])
            xgb_test_val = xgb_results['test']['many_logloss'][
                np.argmin(xgb_results['valid']['many_logloss'])
            ]
            nrounds = np.argmin(xgb_results['valid']['many_logloss']) + 1
            xgb_param = param

        return {'loss': min(xgb_results['valid']['many_logloss']), 'status': STATUS_OK }

    spc = {
        'btype': hp.choice('btype',['I', 'In', 'R', 'Rn']),
        'extra_dims': hp.choice('extra_dims',[0,1,2,3,4,5,6,7]),
        'objective': hp.choice('objective',['manybinary:logistic']),
        'eval_metric':hp.choice('eval_metric',[['many_logloss']]),
        'eta': hp.loguniform('eta', -7, 0),
        'beta_eta': hp.loguniform('beta_eta', -7, 0),
        'max_depth' : hp.choice('max_depth',range(1,11)),
        'subsample': hp.uniform('subsample', 0.5, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
        'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
        'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
        'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
        'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
        'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
    }
    os.environ['HYPEROPT_FMIN_SEED'] = '101010'
    best = fmin(objective,
        space=spc,
        algo=tpe.suggest,
        max_evals=25)
    t1 = time.time()
    results.append({
        'iter': est,
        'valid_logloss': best_val,
        'test_logloss': xgb_test_val,
        'num_rounds': nrounds,
        'time_diff': t1 - t0,
        'winning_param': xgb_param.copy()
    })

{'alpha': 8.813057946902575e-07, 'beta_eta': 0.0016692834513794938, 'btype': 'R', 'colsample_bylevel': 0.7489980469009814, 'colsample_bytree': 0.9420146676089547, 'eta': 0.0070706936561268224, 'eval_metric': ('many_logloss',), 'extra_dims': 2, 'gamma': 0, 'lambda': 0, 'max_depth': 5, 'min_child_weight': 2.1041667775099838e-05, 'objective': 'manybinary:logistic', 'subsample': 0.6262451727083667, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.65380	test-many_logloss:0.65374	valid-many_logloss:0.65379
[200]	train-many_logloss:0.10360	test-many_logloss:0.12882	valid-many_logloss:0.12791
[400]	train-many_logloss:0.03629	test-many_logloss:0.07274	valid-many_logloss:0.07304
[499]	train-many_logloss:0.02289	test-many_logloss:0.06268	valid-many_logloss:0.06328
NEW BEST VALUE!                                       
{'alpha': 0, 'beta_eta': 0.07375799600401638, 'btype': 'I', 'colsample_bylevel': 0.8357246683355697, 'colsample_bytree': 0.7491187395188089, 'eta': 0.605518780660455, 'eval_

../wideboost/wideboost/evals/classification.py:82: RuntimeWarning: divide by zero encountered in log
  return -np.mean(np.log(p))

../wideboost/wideboost/evals/classification.py:80: RuntimeWarning: overflow encountered in exp
  p = 1/(1 + np.exp(-logits))

../wideboost/wideboost/evals/classification.py:82: RuntimeWarning: invalid value encountered in log
  return -np.mean(np.log(p))



{'alpha': 0, 'beta_eta': 0.012588600896345226, 'btype': 'Rn', 'colsample_bylevel': 0.5547269545129563, 'colsample_bytree': 0.9843635186756493, 'eta': 0.5949933323999959, 'eval_metric': ('many_logloss',), 'extra_dims': 4, 'gamma': 0.00013347887526644314, 'lambda': 0, 'max_depth': 9, 'min_child_weight': 7.283920920982167e-06, 'objective': 'manybinary:logistic', 'subsample': 0.9927469091208938, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.94147	test-many_logloss:0.97079	valid-many_logloss:0.96448
  8%|▊         | 2/25 [02:10<21:34, 56.28s/trial, best loss: 0.063283]

../wideboost/wideboost/objectives/manybinarylogloss.py:14: RuntimeWarning: overflow encountered in exp
  P = 1 / (1 + np.exp(-logits))



[19]	train-many_logloss:inf	test-many_logloss:inf	valid-many_logloss:inf
{'alpha': 1.0366863221923632, 'beta_eta': 0.13640103803528691, 'btype': 'Rn', 'colsample_bylevel': 0.6367989155272455, 'colsample_bytree': 0.6974194337846622, 'eta': 0.3029677824305006, 'eval_metric': ('many_logloss',), 'extra_dims': 6, 'gamma': 1.5812851156161036, 'lambda': 1.2783614674881826e-06, 'max_depth': 5, 'min_child_weight': 2.558034001077105, 'objective': 'manybinary:logistic', 'subsample': 0.7481581307597385, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.55526	test-many_logloss:0.55322	valid-many_logloss:0.55533
[20]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
{'alpha': 0.0015268888255807838, 'beta_eta': 0.20179405533498707, 'btype': 'R', 'colsample_bylevel': 0.822829897275416, 'colsample_bytree': 0.8661025998484644, 'eta': 0.011434936316016709, 'eval_metric': ('many_logloss',), 'extra_dims': 1, 'gamma': 0, 'lambda': 2.091554134375915, 'max_depth': 8, 'min_child_weight

...

{'alpha': 0, 'beta_eta': 0.06489049340354942, 'btype': 'I', 'colsample_bylevel': 0.8729133438901646, 'colsample_bytree': 0.5205123603280772, 'eta': 0.0309650866085196, 'eval_metric': ('many_logloss',), 'extra_dims': 5, 'gamma': 0, 'lambda': 7.910525070154376e-06, 'max_depth': 3, 'min_child_weight': 7.405155036744976e-05, 'objective': 'manybinary:logistic', 'subsample': 0.6231629412057431, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.59881	test-many_logloss:0.59815	valid-many_logloss:0.59919
[107]	train-many_logloss:0.01610	test-many_logloss:0.07529	valid-many_logloss:0.08393
{'alpha': 0.014845224338582359, 'beta_eta': 0.03770137496335071, 'btype': 'Rn', 'colsample_bylevel': 0.7211698432094364, 'colsample_bytree': 0.712394975511369, 'eta': 0.008477424619483462, 'eval_metric': ('many_logloss',), 'extra_dims': 1, 'gamma': 0, 'lambda': 0, 'max_depth': 3, 'min_child_weight': 32.756265617720345, 'objective': 'manybinary:logistic', 'subsample': 0.6995489417449295, 'output_dim': 10

{'alpha': 0.12285362381316108, 'beta_eta': 0.0032518959545431397, 'btype': 'R', 'colsample_bylevel': 0.6766284350099434, 'colsample_bytree': 0.8251017236629836, 'eta': 0.1895635060238536, 'eval_metric': ('many_logloss',), 'extra_dims': 3, 'gamma': 0, 'lambda': 0, 'max_depth': 7, 'min_child_weight': 5.941002139647163e-06, 'objective': 'manybinary:logistic', 'subsample': 0.5960145588730401, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.33375	test-many_logloss:0.33517	valid-many_logloss:0.33446
[101]	train-many_logloss:0.00089	test-many_logloss:0.06676	valid-many_logloss:0.07073
100%|██████████| 25/25 [11:34<00:00, 27.80s/trial, best loss: 0.055482]
{'alpha': 8.813057946902575e-07, 'beta_eta': 0.0016692834513794938, 'btype': 'R', 'colsample_bylevel': 0.7489980469009814, 'colsample_bytree': 0.9420146676089547, 'eta': 0.0070706936561268224, 'eval_metric': ('many_logloss',), 'extra_dims': 2, 'gamma': 0, 'lambda': 0, 'max_depth': 5, 'min_child_weight': 2.1041667775099838e-05, 'obje

[0]	train-many_logloss:0.65305	test-many_logloss:0.65297	valid-many_logloss:0.65313
[200]	train-many_logloss:0.04954	test-many_logloss:0.10246	valid-many_logloss:0.09532
[203]	train-many_logloss:0.04886	test-many_logloss:0.10262	valid-many_logloss:0.09546
{'alpha': 2.7324514268482383, 'beta_eta': 0.08036958137175818, 'btype': 'R', 'colsample_bylevel': 0.7987531851832483, 'colsample_bytree': 0.7299740823920091, 'eta': 0.06030778497723384, 'eval_metric': ('many_logloss',), 'extra_dims': 3, 'gamma': 0, 'lambda': 0, 'max_depth': 8, 'min_child_weight': 0.00023882505207060682, 'objective': 'manybinary:logistic', 'subsample': 0.8750385697829414, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.42196	test-many_logloss:0.42207	valid-many_logloss:0.42207
[65]	train-many_logloss:0.00069	test-many_logloss:0.06019	valid-many_logloss:0.05699
NEW BEST VALUE!                                                        
{'alpha': 0, 'beta_eta': 0.009060610135766442, 'btype': 'I', 'colsample_bylevel'

[0]	train-many_logloss:0.65198	test-many_logloss:0.65196	valid-many_logloss:0.65212
[200]	train-many_logloss:0.10164	test-many_logloss:0.12623	valid-many_logloss:0.12873
[400]	train-many_logloss:0.03322	test-many_logloss:0.06992	valid-many_logloss:0.07124
[499]	train-many_logloss:0.02014	test-many_logloss:0.06035	valid-many_logloss:0.06149
NEW BEST VALUE!                                       
{'alpha': 0, 'beta_eta': 0.07375799600401638, 'btype': 'I', 'colsample_bylevel': 0.8357246683355697, 'colsample_bytree': 0.7491187395188089, 'eta': 0.605518780660455, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0, 'max_depth': 5, 'min_child_weight': 118.844312919608, 'objective': 'manybinary:logistic', 'subsample': 0.6525233714163394, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.39745	test-many_logloss:0.39657	valid-many_logloss:0.39779
[79]	train-many_logloss:0.09116	test-many_logloss:0.13035	valid-many_logloss:0.13504
{'alpha': 0, 'beta_eta': 0.012588600

[66]	train-many_logloss:0.00065	test-many_logloss:0.05543	valid-many_logloss:0.05546
NEW BEST VALUE!                                                        
{'alpha': 0, 'beta_eta': 0.009060610135766442, 'btype': 'I', 'colsample_bylevel': 0.7332732947918047, 'colsample_bytree': 0.5030735772414907, 'eta': 0.0015245875835189877, 'eval_metric': ('many_logloss',), 'extra_dims': 7, 'gamma': 9.066632495478617e-06, 'lambda': 0, 'max_depth': 3, 'min_child_weight': 135.18110003620356, 'objective': 'manybinary:logistic', 'subsample': 0.5593100367488764, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.68699	test-many_logloss:0.68698	valid-many_logloss:0.68699
[200]	train-many_logloss:0.16255	test-many_logloss:0.16991	valid-many_logloss:0.17175
[400]	train-many_logloss:0.08542	test-many_logloss:0.10051	valid-many_logloss:0.10075
[499]	train-many_logloss:0.07086	test-many_logloss:0.09094	valid-many_logloss:0.09025
{'alpha': 9.442657203270879e-05, 'beta_eta': 0.17396370377116593, 'btype': '

In [7]:
import pandas as pd
pd.concat([
    pd.Series(r).to_frame().T for r in results
], axis=0).to_csv('./mnist_wxgb_results_paper_2.csv', index=False)